In [ ]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv

--2023-03-21 17:53:15--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-21 17:53:15--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv.1’

dataset_ozon.tsv.1  100%[===================>]   9.19M  --.-KB/s    in 0.04s   

2023-03-21 17:53:16 (218 MB/s) - ‘dataset_ozon.tsv.1’ saved [9640627/9640627]

FINISHED --2023-03-21 17:53:16--
Total wall clock time: 0.5s
Downloaded: 1 files, 9.2M in 0.04s (218 MB/s)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t', encoding='utf-8')
ds = ds.drop(columns = ['Unnamed: 0'])

In [ ]:
def make_tags(n, b):
    global tags
    ner_tags_id = []
    name = wordpunct_tokenize(n)
    brand = wordpunct_tokenize(b)
    results=[]
    sll=len(brand)
    for ind in (i for i,e in enumerate(name) if e==brand[0]):
        if name[ind:ind+sll]==brand:
            results.append((ind,ind+sll-1))
    if results != []:
      ner_tags_labels = ['O'] * len(name)  
      ner_tags_id = [0] * len(name)     
      for result in results:
          if result[0] == result[1]:
              ner_tags_labels[result[0]] = 'B-BRAND'
              ner_tags_id[result[0]] = 1
          else:
            ner_tags_labels[result[0]] = 'B-BRAND'
            ner_tags_id[result[0]] = 1
            for j in range(result[0]+1, result[1]+1):
              ner_tags_labels[j] = 'I-BRAND'
              ner_tags_id[j] = 2

      tags.append({'tokens': name, 'ner_tags_labels': ner_tags_labels, 'ner_tags_id': ner_tags_id})

In [ ]:
tags = []
x = 0
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [00:02<00:00, 29211.96it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
81774    None
81775    None
81776    None
81777    None
81778    None
Length: 81779, dtype: object

In [ ]:
ds_IOB2 = pd.DataFrame(tags, columns = ['tokens', 'ner_tags_labels', 'ner_tags_id'])

In [ ]:
!pip install datasets
!pip install transformers datasets evaluate seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import Dataset, Features, Value, ClassLabel,  Sequence
ner_tags_labels_ = ['O', 'B-BRAND', 'I-BRAND']
ds_features = Features({'tokens':  Sequence(Value("string")),
                        'ner_tags_labels': Sequence(ClassLabel(names=ner_tags_labels_)),
                        'ner_tags_id': Sequence(Value("int64"))})

dataset = Dataset.from_pandas(ds_IOB2, features=ds_features)
dataset_splitted = dataset.train_test_split(test_size=0.2, seed=88)

In [ ]:
dataset

Dataset({
    features: ['tokens', 'ner_tags_labels', 'ner_tags_id'],
    num_rows: 81527
})

In [ ]:
import numpy as np
import torch


from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
ds_tokenized = dataset_splitted.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/65221 [00:00<?, ? examples/s]

Map:   0%|          | 0/16306 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")
label_list = ds_tokenized["train"].features["ner_tags_labels"].feature.names

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Игнорируем любые предсказания на служебных токенах при подсчете метрик
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {i: label_list[i] for i in range(len(label_list))}
label2id = {label_list[i]: i for i in range(len(label_list))}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
training_args = TrainingArguments(
    output_dir="xlm_roberta_base_brand_rus",
    learning_rate=4e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.100900,0.046572,0.894343,0.902562,0.898434,0.985327
1000,0.054200,0.041213,0.909483,0.925202,0.917275,0.987789
1500,0.044500,0.033831,0.929779,0.930741,0.930259,0.990151
2000,0.036100,0.036960,0.925879,0.935853,0.930839,0.990019
2500,0.036100,0.030674,0.940321,0.944556,0.942434,0.991636
3000,0.032400,0.027626,0.949694,0.953624,0.951655,0.992860
3500,0.027300,0.025644,0.950968,0.953746,0.952355,0.993003
4000,0.028100,0.024724,0.951035,0.953929,0.952479,0.993164
4500,0.022000,0.022467,0.953634,0.957580,0.955603,0.993686
5000,0.018200,0.021603,0.959752,0.959284,0.959518,0.994141


TrainOutput(global_step=8154, training_loss=0.030312790790550374, metrics={'train_runtime': 2970.2401, 'train_samples_per_second': 43.916, 'train_steps_per_second': 2.745, 'total_flos': 3170109492560880.0, 'train_loss': 0.030312790790550374, 'epoch': 2.0})